In [1]:
# Se importan las librerias
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from Funciones.GEO_LIB import Plot, Poligono, Distancia, Geodecodificacion


**1. Lectura de datos**

In [2]:
datos=pd.read_csv('Base de datos/datos_lematizados.csv')
datos['lemmatized']=datos['lemmatized'].astype(str)
datos = datos.drop(datos.columns[0], axis=1)

In [3]:
# Filtra el DataFrame aquellas filas que tienen comentarios con espacios vacíos
df_filtered = datos[~datos['lemmatized'].str.contains(r'^\s*$')]
df_filtered.head()

,Unnamed: 0,index,Id,placeId,place name,Categoria query,Year,text,stars,Idioma,traduccion,nps,lemmatized
0,0,2,2,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"Hermoso atractivo turistico, cuanta con sender...",5.0,es,hermoso atractivo turistico cuanta con sendero...,promotor,hermoso atractivo turistico cuantar con sender...
1,1,4,4,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"Muy lindo parque nacional, 100% recomendado, m...",5.0,es,muy lindo parque nacional recomendado muy lind...,promotor,mucho lindo parque nacional recomendado mucho ...
2,2,5,5,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,Excelentes paisajes. Un recorrido muy pulcro y...,5.0,es,excelentes paisajes un recorrido muy pulcro y ...,promotor,excelente paisaje uno recorrido mucho pulcro y...
3,3,6,6,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"hermoso, vengo a este lugar desde el año 1968....",4.0,es,hermoso vengo a este lugar desde el ano lament...,promotor,hermoso venir a este lugar desde el ano lament...
4,4,8,8,ChIJA7e1akBiF5YRGX2D8me8Pnw,Petrohué Waterfalls,tourist attraction,2023,"Idiotically crowded, low water levels, mainly ...",1.0,en,estupidamente abarrotado bajos niveles de agua...,detractor,estupidamente abarrotado bajo nivel de agua pr...


**Sistema de recomendación**

In [4]:
# Se lee la matriz de similaridad
coseno_sim = np.load('Resultados/ Matriz_similaridad.npy')

In [5]:
# Se crea una lista ordenada de los nombres de los lugares según el indice de placeId en la matriz de frecuencia,
# de modo que el índice de la serie pueda coincidir con el índice de fila y columna de la matriz de similitud.
indices = df_filtered[['placeId','place name','Categoria query']].drop_duplicates().sort_values('placeId').reset_index(drop='index')
indices.head()

,placeId,place name,Categoria query
0,ChIJ-01C_3EZIpYR9UpdwV9D6Y8,Cabala Bar,restaurant
1,ChIJ-0drS5c7GJYRycRzXHMYzpU,Plaza Mirasol,park
2,ChIJ-0eSrjUTIpYRFlqkSZzyfF4,Cruce Mocopulli,transportation service
3,ChIJ-10I8a07GJYRfzQAOYA0vPA,Hostal Pacífico,hotel
4,ChIJ-1zGXfziF5YRJVZTNNO4K1Y,Plaza de Armas de Fresia,national park


In [6]:
datos_coordenadas = pd.read_excel('Base de datos/Base de datos Chiloe Full.xlsx').reset_index()
datos_coordenadas = datos_coordenadas.rename({'index':'Id','latitude':'Latitud','longitude':'Longitud'}, axis='columns')


In [7]:
datos_coordenadas = datos_coordenadas.merge(indices[['placeId','Categoria query']],on='placeId')

In [8]:

def recomendar_lugares(lugar, coseno_sim, indices, datos, top_n=10):
    """
    Función que recomienda lugares similares a uno dado.

    Parameters:
        lugar (str): El nombre del lugar para el que deseas obtener recomendaciones.
        coseno_sim (pd.DataFrame): La matriz de similitud coseno entre lugares.
        indices (pd.DataFrame): Un DataFrame que mapea los nombres de lugares a índices.
        top_n (int): El número de lugares recomendados a devolver (por defecto, 10).

    Returns:
        pd.DataFrame: Un DataFrame que contiene los lugares recomendados y sus índices de similitud.
    """
    lugares_recomendados = []

    if lugar not in indices['place name'].values:
        return pd.DataFrame(columns=['place name', 'similarity'])  # Devolver DataFrame vacío

    idx = indices[indices['place name'] == lugar].index[0]
    sim_scores = list(enumerate(coseno_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]  # Excluye el lugar en sí mismo y toma los mejores 'top_n' lugares
    lugares_recomendados.append((lugar,1,lugar,14,1))
    for i, score in sim_scores:
        lugar_recomendado = indices.loc[i, 'place name']
        lugares_recomendados.append((lugar_recomendado, score, lugar_recomendado, 14, 2))
        
    # Convierte la lista de tuplas en un DataFrame
    recomendaciones_df = pd.DataFrame(lugares_recomendados, columns=['place name', 'similarity','Label','tamano','Color'])
    recomendaciones_df = recomendaciones_df.merge(datos[['place name','Latitud','Longitud','Categoria query','reviews']].drop_duplicates('place name'),on='place name')
    
    Plot().Latlon(df = recomendaciones_df,scalecolor='viridis',file_name='Resultados/'+lugar+'.html')
    recomendaciones_df = recomendaciones_df.drop(columns=['Label','tamano'],axis=1)

    return recomendaciones_df


In [9]:
# Ejemplo de uso
lugares_similares = recomendar_lugares('Acceso a la Playa de Quincho Mutrico', coseno_sim, indices,datos_coordenadas)
lugares_similares


,place name,similarity,Color,Latitud,Longitud,Categoria query,reviews
0,Acceso a la Playa de Quincho Mutrico,1.000000,1,-41.873626,-73.780967,national park,22
1,Terminal Chaiten,0.373049,2,-42.915833,-72.711439,bus station,126
2,Restaurant y Hospedaje Costa Azul,0.354049,2,-41.769876,-73.126302,restaurant,135
3,Lago Llanquihue,0.349562,2,-41.135050,-72.820436,national park,1212
4,Maicolpué,0.348630,2,-40.595163,-73.734681,beach,2794
5,"Cabañas y camping Frente al Mar San Javier, Cu...",0.341688,2,-42.399454,-73.644412,hotel,42
6,Sabores Dalcahue,0.340686,2,-42.381107,-73.649554,restaurant,27
7,Monumento a la Colonización Alemana,0.339646,2,-41.472289,-72.938657,tourist attraction,27
8,"Playa Mar Brava, Carelmapu.",0.338721,2,-41.740937,-73.734493,beach,603
9,Apart Hotel Don Teodoro,0.336960,2,-41.865596,-73.827487,hotel,59


**A continuación se probará el modelo:**

In [10]:
# Ejemplo de uso
lugares_similares = recomendar_lugares('Chiloé National Park', coseno_sim, indices,datos_coordenadas)
lugares_similares

,place name,similarity,Color,Latitud,Longitud,Categoria query,reviews
0,Chiloé National Park,1.000000,1,-42.422888,-74.082950,national park,436
1,Koopa Bar,0.770322,2,-40.787686,-73.201229,pizza restaurant,29
2,La Madre de las Masas,0.759232,2,-41.461589,-72.935889,pizza restaurant,10
3,Rukaffee Tienda Virtual,0.748349,2,-41.470250,-72.947590,cafeteria,473
4,RESTAURANT CABAÑAS RUTA 215,0.729118,2,-40.590607,-72.944225,restaurant,38
5,Cabañas volcanes de frutilar,0.720846,2,-41.120859,-73.045887,volcano,52
6,Parque Bosque De Bella vista,0.717591,2,-40.594173,-73.104508,park,98
7,Parque Licanrayen,0.709427,2,-41.326119,-72.988285,park,74
8,Cabañas Aitue,0.706253,2,-43.187469,-71.868653,hotel,80
9,Parque IV Centenario,0.705090,2,-40.568637,-73.138827,national park,236


In [11]:
# Ejemplo de uso
lugares_similares = recomendar_lugares('Iglesia de San Francisco, Castro', coseno_sim, indices,datos_coordenadas)
lugares_similares

,place name,similarity,Color,Latitud,Longitud,Categoria query,reviews
0,"Iglesia de San Francisco, Castro",1.000000,1,-42.481626,-73.763691,church,6158
1,Pub La Vitrola Osorno,0.613608,2,-40.573805,-73.140124,cafeteria,319
2,Nama Ramen,0.559305,2,-41.324023,-72.984384,restaurant,124
3,Centro De Eventos La Negra,0.537253,2,-41.773832,-73.171934,casino,34
4,Ecoturismo Puyehue,0.527460,2,-40.798107,-72.322725,hotel,12
5,Cabañas Los Fabianes,0.522553,2,-42.565285,-73.820794,hotel,48
6,El Sitio Restaurante,0.516854,2,-41.215053,-72.549545,restaurant,95
7,Residencial Rega,0.515551,2,-40.578336,-73.131906,hotel,32
8,Nómades,0.510853,2,-42.501417,-73.787132,hotel,80
9,Taberna Nos,0.507110,2,-43.116416,-73.613988,restaurant,263


In [12]:
lugares_similares = recomendar_lugares('Mesa Tropera', coseno_sim, indices,datos_coordenadas)
lugares_similares

,place name,similarity,Color,Latitud,Longitud,Categoria query,reviews
0,Mesa Tropera,1.000000,1,-41.314537,-72.981764,cafeteria,7782
1,Paihuen Yurts & turismo,0.867673,2,-41.893008,-73.883410,tourist attraction,28
2,Hotel Colonial,0.859904,2,-41.768867,-73.127082,hotel,133
3,Chucao Restaurant,0.856954,2,-41.802028,-72.719854,cafeteria,65
4,Café Chiloense,0.851104,2,-42.664540,-74.012091,cafeteria,23
5,Chiloe Silvestre,0.851051,2,-41.839786,-73.936015,wildlife refuge,22
6,Parque Bellavista,0.844133,2,-40.582465,-73.154449,park,1515
7,Restaurante Playa Dalcahue,0.827811,2,-42.380188,-73.647965,restaurant,189
8,Cabañas Los Cisnes De Pucatue,0.814621,2,-42.641754,-73.753170,hotel,35
9,Hostal Climbhouse,0.814354,2,-41.328507,-72.956797,hotel,145


In [13]:
lugares_similares = recomendar_lugares('Muelle de Frutillar', coseno_sim, indices,datos_coordenadas)
lugares_similares

,place name,similarity,Color,Latitud,Longitud,Categoria query,reviews
0,Muelle de Frutillar,1.000000,1,-41.135894,-73.026523,island,12020
1,Cabañas Estero San Juan,0.453478,2,-42.421390,-73.745366,hotel,59
2,"HOSPEDAJE ""LA CHALIA""",0.369857,2,-42.676043,-73.899237,hotel,11
3,Café del Lago Huillinco,0.358281,2,-42.672866,-73.897145,restaurant,380
4,Centro de Arte Molino Machmar,0.342710,2,-41.317532,-72.993719,performing arts theater,581
5,"Playa Rio Sur ,Maicolpue",0.331349,2,-40.603805,-73.743720,beach,697
6,Zaferani Sabor de Oriente Puerto Varas,0.322635,2,-41.319186,-72.985768,cafeteria,30
7,Hostal señora marta,0.317408,2,-40.684398,-72.599897,hostel,15
8,Yatehue Hotel & Tour,0.316068,2,-42.889681,-73.474120,beach,62
9,783 restobar,0.310693,2,-42.380089,-73.652928,restaurant,88


**A continuación se analizarán los comentarios que contienen a la palabra Rilar**

In [14]:
filas_con_rilar = datos[datos['lemmatized'].str.contains(' rilar', case=False, na=False)]
filas_con_rilar [['lemmatized']]

,lemmatized
112482,iglesia ubicado en rilar en el isla grande de ...
112485,festividad de el iglesia santa maria de rilar ...
112488,hermoso iglesia caracteristico del territorio ...
112492,linda iglesia en el peninsula de rilar
112493,el iglesia de rilar deber visitar él
112496,lindo obra arquitectonico de rilar chiloe
112513,maravilloso arquitectura de rilar
112514,linda iglesia en rilar
112525,iglesia patrimonial de rilar
112582,bello iglesia catolico ubicado en el sector de...


**Cuando se lematizan las palabras, la palabra 'Rilan' se transforma en 'Rilar**